In [152]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import glob
import re
import os

# Disable pandas warning
import warnings
warnings.filterwarnings('ignore')

In [153]:
#dataset_prefix = "MSC"
dataset_prefix = "TC"


files = glob.glob(f'processed/{dataset_prefix}/*.xlsx')
print(files)
metrics = [re.match(r'.*/.*/(.*).xlsx', f).group(1) for f in files]
print(metrics)
# remove Bert-f1
# metrics.remove('Bert-f1')

output_dir = f"ROI_v2/{dataset_prefix}"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

['processed/TC/Bert-f1.xlsx', 'processed/TC/BLEU.xlsx', 'processed/TC/Bleurt.xlsx', 'processed/TC/DEB.xlsx', 'processed/TC/METEOR.xlsx', 'processed/TC/rougeL.xlsx']
['Bert-f1', 'BLEU', 'Bleurt', 'DEB', 'METEOR', 'rougeL']


In [154]:
if dataset_prefix == "MSC":
    # title_fontsize = 36
    legend_fontsize = 18
    label_fontsize = 24
else:
    # title_fontsize = 
    legend_fontsize = 20
    label_fontsize = 20

# Set to a common linux font
fonts = {'family' : 'serif',
        'serif' : 'Caladea',
        'weight' : 'bold',
        'size'   : 36
        }
# For some wierd reason you need to set the 
# font size like following
# 36 for MSC
# 40 for TC 

matplotlib.rc('font', **fonts)

In [155]:
# Load all sheets
global_df = pd.DataFrame()
global_budget_df = pd.DataFrame()
num_sheets_read = 0
for fpath, metric in zip(files, metrics):
    sheets = pd.ExcelFile(fpath).sheet_names
    for sheet in sheets:
        # Only ROI sheets
        print(fpath, sheet)
        if sheet.endswith('_ROI') and not sheet.startswith('merged'):

            # Read budget
            budget_sheet = sheet.replace('_ROI', '_budget')
            budget_df = pd.read_excel(fpath, sheet_name=budget_sheet)
            if budget_df.shape[0] == 1:
                # Skip this!
                continue
            budget_df['experiment'] = sheet
            budget_df['metric'] = "total"
            global_budget_df = global_budget_df.append(budget_df, ignore_index=True)

            ibudget_sheet = sheet.replace('_ROI', '_ibudget')
            ibudget_df = pd.read_excel(fpath, sheet_name=ibudget_sheet)
            ibudget_df['experiment'] = sheet
            ibudget_df['metric'] = "input"
            global_budget_df = global_budget_df.append(ibudget_df, ignore_index=True)

            num_sheets_read += 1

            # Read metric.xlsx
            df = pd.read_excel(fpath, sheet_name=sheet)

            # Remove " (ppl)" from column name and add it as a column
            df.columns = [re.sub(r' \(ppl\)', '', c) for c in df.columns]

            # Remove unnamed columns
            df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

            df['metric'] = metric
            df['experiment'] = sheet
            global_df = global_df.append(df, ignore_index=True)

            

processed/TC/Bert-f1.xlsx None_normal
processed/TC/Bert-f1.xlsx None_normal_budget
processed/TC/Bert-f1.xlsx None_normal_ibudget
processed/TC/Bert-f1.xlsx None_normal_ROI
processed/TC/Bert-f1.xlsx None_ppl
processed/TC/Bert-f1.xlsx None_ppl_budget
processed/TC/Bert-f1.xlsx None_ppl_ibudget
processed/TC/Bert-f1.xlsx None_ppl_ROI
processed/TC/Bert-f1.xlsx Pegasus-CD_normal
processed/TC/Bert-f1.xlsx Pegasus-CD_normal_budget
processed/TC/Bert-f1.xlsx Pegasus-CD_normal_ibudget
processed/TC/Bert-f1.xlsx Pegasus-CD_normal_ROI
processed/TC/Bert-f1.xlsx Pegasus-CD_ppl
processed/TC/Bert-f1.xlsx Pegasus-CD_ppl_budget
processed/TC/Bert-f1.xlsx Pegasus-CD_ppl_ibudget
processed/TC/Bert-f1.xlsx Pegasus-CD_ppl_ROI
processed/TC/Bert-f1.xlsx merged_ppl
processed/TC/Bert-f1.xlsx merged_ppl_ROI
processed/TC/Bert-f1.xlsx merged_normal
processed/TC/Bert-f1.xlsx merged_normal_ROI
processed/TC/BLEU.xlsx None_normal
processed/TC/BLEU.xlsx None_normal_budget
processed/TC/BLEU.xlsx None_normal_ibudget
processed/

In [156]:
num_sheets_read

24

In [157]:
global_df

,History Signal,flanT5-XL FS-ICL,flanT5-XL ZS-ICL,T0 FS-ICL,T0 ZS-ICL,Tk-Instruct FS-ICL,Tk-Instruct ZS-ICL,GPT-3 FS-ICL,GPT-3 ZS-ICL,metric,experiment
0,BART,0.024533,0.193281,-0.041131,-0.156123,0.145000,1.582288,-0.755247,0.137542,Bert-f1,None_normal_ROI
1,Full,0.417784,0.937980,0.446438,0.714167,0.411378,1.650446,0.644145,1.446531,Bert-f1,None_normal_ROI
2,Pegasus-CD,-0.059001,0.148503,-0.350821,-0.742770,-0.198064,1.675560,-0.200736,-0.086305,Bert-f1,None_normal_ROI
3,PegasusFT,0.007527,0.139955,-0.540529,-1.141289,0.115919,1.653232,-0.266032,-0.060294,Bert-f1,None_normal_ROI
4,Recent-1,-1.240383,-0.859228,-0.131141,-0.584085,-0.805896,2.089213,NaN,NaN,Bert-f1,None_normal_ROI
...,...,...,...,...,...,...,...,...,...,...,...
193,Semantic-10,-0.659679,0.537217,0.015267,0.729356,-0.068992,2.890881,NaN,NaN,rougeL,None_ppl_ROI
194,None_1,NaN,NaN,NaN,NaN,NaN,NaN,-0.588925,-0.797600,rougeL,Pegasus-CD_normal_ROI
195,PegasusFT,-0.601918,-0.277831,-0.421820,-0.462224,-0.377277,1.410085,-0.090034,-0.246360,rougeL,Pegasus-CD_normal_ROI
196,None_1,-0.609587,-0.338313,-0.396797,-0.379684,-0.359188,1.407481,NaN,NaN,rougeL,Pegasus-CD_ppl_ROI


In [158]:
global_df.shape, global_budget_df.shape

((198, 11), (456, 19))

In [159]:
global_budget_df

,History Signal,flanT5-XL FS-ICL,flanT5-XL ZS-ICL,T0 FS-ICL,T0 ZS-ICL,Tk-Instruct FS-ICL,Tk-Instruct ZS-ICL,GPT-3 FS-ICL,GPT-3 ZS-ICL,experiment,metric,flanT5-XL FS-ICL (ppl),flanT5-XL ZS-ICL (ppl),T0 FS-ICL (ppl),T0 ZS-ICL (ppl),Tk-Instruct FS-ICL (ppl),Tk-Instruct ZS-ICL (ppl),GPT-3 FS-ICL (ppl),GPT-3 ZS-ICL (ppl)
0,None_0,146.109542,60.708923,158.969669,63.803759,142.574648,52.539916,167.520333,86.116629,None_normal_ROI,total,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,None_1,146.109542,60.708923,158.969669,63.803759,142.574648,52.539916,167.520333,86.116629,None_normal_ROI,total,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,BART,312.429138,129.771317,301.391947,142.083734,280.893684,121.836941,349.080438,175.441030,None_normal_ROI,total,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Full,833.002895,408.683948,835.389988,409.662435,828.717976,410.516569,832.218555,421.953234,None_normal_ROI,total,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Pegasus-CD,314.133707,143.575295,335.821308,160.530020,312.000668,138.423000,346.203545,183.355559,None_normal_ROI,total,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
451,None_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pegasus-CD_ppl_ROI,total,827.305973,413.364004,861.003296,450.575494,838.999690,415.861081,NaN,NaN
452,PegasusFT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pegasus-CD_ppl_ROI,total,831.825627,400.422832,842.755657,430.586050,866.970021,412.978042,NaN,NaN
453,None_0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pegasus-CD_ppl_ROI,input,131.657069,47.182548,131.653617,47.146713,131.655086,47.147093,131.657069,47.182548
454,None_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pegasus-CD_ppl_ROI,input,793.270596,374.393717,793.272582,374.391324,793.273693,374.391324,NaN,NaN


In [160]:
global_df

,History Signal,flanT5-XL FS-ICL,flanT5-XL ZS-ICL,T0 FS-ICL,T0 ZS-ICL,Tk-Instruct FS-ICL,Tk-Instruct ZS-ICL,GPT-3 FS-ICL,GPT-3 ZS-ICL,metric,experiment
0,BART,0.024533,0.193281,-0.041131,-0.156123,0.145000,1.582288,-0.755247,0.137542,Bert-f1,None_normal_ROI
1,Full,0.417784,0.937980,0.446438,0.714167,0.411378,1.650446,0.644145,1.446531,Bert-f1,None_normal_ROI
2,Pegasus-CD,-0.059001,0.148503,-0.350821,-0.742770,-0.198064,1.675560,-0.200736,-0.086305,Bert-f1,None_normal_ROI
3,PegasusFT,0.007527,0.139955,-0.540529,-1.141289,0.115919,1.653232,-0.266032,-0.060294,Bert-f1,None_normal_ROI
4,Recent-1,-1.240383,-0.859228,-0.131141,-0.584085,-0.805896,2.089213,NaN,NaN,Bert-f1,None_normal_ROI
...,...,...,...,...,...,...,...,...,...,...,...
193,Semantic-10,-0.659679,0.537217,0.015267,0.729356,-0.068992,2.890881,NaN,NaN,rougeL,None_ppl_ROI
194,None_1,NaN,NaN,NaN,NaN,NaN,NaN,-0.588925,-0.797600,rougeL,Pegasus-CD_normal_ROI
195,PegasusFT,-0.601918,-0.277831,-0.421820,-0.462224,-0.377277,1.410085,-0.090034,-0.246360,rougeL,Pegasus-CD_normal_ROI
196,None_1,-0.609587,-0.338313,-0.396797,-0.379684,-0.359188,1.407481,NaN,NaN,rougeL,Pegasus-CD_ppl_ROI


In [161]:
global_df["History Signal"].value_counts()

PegasusFT      24
None_1         18
BART           12
Full           12
Pegasus-CD     12
Recent-1       12
Recent-2       12
Recent-4       12
Recent-8       12
Recent-10      12
Semantic-1     12
Semantic-2     12
Semantic-4     12
Semantic-8     12
Semantic-10    12
Name: History Signal, dtype: int64

In [162]:
global_budget_df["History Signal"].value_counts()/2

None_0         24.0
None_1         24.0
PegasusFT      24.0
BART           12.0
Full           12.0
Pegasus-CD     12.0
Recent-1       12.0
Recent-2       12.0
Recent-4       12.0
Recent-8       12.0
Recent-10      12.0
Semantic-1     12.0
Semantic-2     12.0
Semantic-4     12.0
Semantic-8     12.0
Semantic-10    12.0
Name: History Signal, dtype: float64

In [163]:
# Check if everything aligns between global_df and global_budget_df
#sub_budget_df = global_budget_df.loc[(global_budget_df["History Signal"] != "None_0") & (global_budget_df["metric"] == "input")]
# Values in "experiment" columns are exact match, but index is different
#assert (global_df.reset_index()["experiment"] == sub_budget_df.reset_index()["experiment"]).all()
# "History Signal" columns are exact match, but index is different
#assert (global_df.reset_index()["History Signal"] == sub_budget_df.reset_index()["History Signal"]).all()

In [164]:
models = global_df.columns.tolist()
models.remove('metric')
models.remove('experiment')
models.remove('History Signal')

retained_cols = ['History Signal', 'metric', 'experiment']

rotated_df = pd.DataFrame()
for c in models:
    sub_df = global_df.loc[:, retained_cols + [c]]
    sub_df['model'] = c
    # Make each type of metric to a column
    sub_df = sub_df.pivot_table(index=['History Signal', 'model', 'experiment'], columns='metric', values=c)

    rotated_df = rotated_df.append(sub_df, ignore_index=False)

rotated_df = rotated_df.reset_index()

In [165]:
rotated_df

metric,History Signal,model,experiment,BLEU,Bert-f1,Bleurt,DEB,METEOR,rougeL
0,BART,flanT5-XL FS-ICL,None_normal_ROI,0.013192,0.024533,0.181337,2.827780,-0.012220,-0.017885
1,BART,flanT5-XL FS-ICL,None_ppl_ROI,0.649517,-1.284791,2.342523,1.832087,1.632054,-0.564694
2,Full,flanT5-XL FS-ICL,None_normal_ROI,0.266205,0.417784,0.712758,3.268328,0.721557,-0.006995
3,Full,flanT5-XL FS-ICL,None_ppl_ROI,0.279320,0.427779,0.923966,3.425458,0.861939,0.039829
4,None_1,flanT5-XL FS-ICL,None_ppl_ROI,2.941883,-6.931000,9.561130,-30.052959,4.687062,-2.207612
...,...,...,...,...,...,...,...,...,...
203,Pegasus-CD,GPT-3 ZS-ICL,None_normal_ROI,-0.033461,-0.086305,0.052354,5.184283,1.639011,-0.284675
204,PegasusFT,GPT-3 ZS-ICL,None_normal_ROI,-0.123567,-0.060294,0.762640,3.414238,2.464788,-0.367376
205,PegasusFT,GPT-3 ZS-ICL,Pegasus-CD_normal_ROI,-0.149586,-0.152124,-0.596416,-2.602414,0.830964,-0.246360
206,Recent-4,GPT-3 ZS-ICL,None_normal_ROI,-0.022504,0.227191,0.728828,4.130606,1.423846,-0.131969


In [166]:
global_budget_df

,History Signal,flanT5-XL FS-ICL,flanT5-XL ZS-ICL,T0 FS-ICL,T0 ZS-ICL,Tk-Instruct FS-ICL,Tk-Instruct ZS-ICL,GPT-3 FS-ICL,GPT-3 ZS-ICL,experiment,metric,flanT5-XL FS-ICL (ppl),flanT5-XL ZS-ICL (ppl),T0 FS-ICL (ppl),T0 ZS-ICL (ppl),Tk-Instruct FS-ICL (ppl),Tk-Instruct ZS-ICL (ppl),GPT-3 FS-ICL (ppl),GPT-3 ZS-ICL (ppl)
0,None_0,146.109542,60.708923,158.969669,63.803759,142.574648,52.539916,167.520333,86.116629,None_normal_ROI,total,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,None_1,146.109542,60.708923,158.969669,63.803759,142.574648,52.539916,167.520333,86.116629,None_normal_ROI,total,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,BART,312.429138,129.771317,301.391947,142.083734,280.893684,121.836941,349.080438,175.441030,None_normal_ROI,total,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Full,833.002895,408.683948,835.389988,409.662435,828.717976,410.516569,832.218555,421.953234,None_normal_ROI,total,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Pegasus-CD,314.133707,143.575295,335.821308,160.530020,312.000668,138.423000,346.203545,183.355559,None_normal_ROI,total,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
451,None_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pegasus-CD_ppl_ROI,total,827.305973,413.364004,861.003296,450.575494,838.999690,415.861081,NaN,NaN
452,PegasusFT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pegasus-CD_ppl_ROI,total,831.825627,400.422832,842.755657,430.586050,866.970021,412.978042,NaN,NaN
453,None_0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pegasus-CD_ppl_ROI,input,131.657069,47.182548,131.653617,47.146713,131.655086,47.147093,131.657069,47.182548
454,None_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pegasus-CD_ppl_ROI,input,793.270596,374.393717,793.272582,374.391324,793.273693,374.391324,NaN,NaN


In [167]:
std = global_budget_df.groupby(['History Signal', 'experiment', 'metric']).std()
# STD should either be zero or nan
std = std.fillna(0)
std = std.reset_index()
std
# assert (std['History Signal'] == rotated_df['History Signal']).all()

,History Signal,experiment,metric,flanT5-XL FS-ICL,flanT5-XL ZS-ICL,T0 FS-ICL,T0 ZS-ICL,Tk-Instruct FS-ICL,Tk-Instruct ZS-ICL,GPT-3 FS-ICL,GPT-3 ZS-ICL,flanT5-XL FS-ICL (ppl),flanT5-XL ZS-ICL (ppl),T0 FS-ICL (ppl),T0 ZS-ICL (ppl),Tk-Instruct FS-ICL (ppl),Tk-Instruct ZS-ICL (ppl),GPT-3 FS-ICL (ppl),GPT-3 ZS-ICL (ppl)
0,BART,None_normal_ROI,input,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,BART,None_normal_ROI,total,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,BART,None_ppl_ROI,input,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,BART,None_ppl_ROI,total,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Full,None_normal_ROI,input,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,Semantic-4,None_ppl_ROI,total,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
72,Semantic-8,None_normal_ROI,input,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
73,Semantic-8,None_normal_ROI,total,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
74,Semantic-8,None_ppl_ROI,input,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [168]:
# Remove redundant rows (because budget rows are repeated for each metric)
global_budget_df = global_budget_df.groupby(['History Signal', 'experiment', 'metric']).first().reset_index()

In [169]:
# Pivoting
models = global_budget_df.columns.tolist()
models.remove('metric')
models.remove('experiment')
models.remove('History Signal')

retained_cols = ['History Signal', 'experiment', 'metric']

rotated_budget_df = pd.DataFrame()
for c in models:
    sub_df = global_budget_df.loc[:, retained_cols + [c]]
    sub_df['model'] = c
    # Make each type of metric to a column
    sub_df = sub_df.pivot_table(index=['History Signal', 'model', 'experiment'], columns='metric', values=c)

    rotated_budget_df = rotated_budget_df.append(sub_df, ignore_index=False)

rotated_budget_df.reset_index(inplace=True)

In [170]:
rotated_budget_df

metric,History Signal,model,experiment,input,total
0,BART,flanT5-XL FS-ICL,None_normal_ROI,298.001292,312.429138
1,Full,flanT5-XL FS-ICL,None_normal_ROI,791.706841,833.002895
2,None_0,flanT5-XL FS-ICL,None_normal_ROI,131.657069,146.109542
3,None_0,flanT5-XL FS-ICL,Pegasus-CD_normal_ROI,131.657069,146.109542
4,None_1,flanT5-XL FS-ICL,None_normal_ROI,131.657069,146.109542
...,...,...,...,...,...
243,Semantic-8,Tk-Instruct ZS-ICL (ppl),None_ppl_ROI,243.369054,274.161901
244,None_0,GPT-3 FS-ICL (ppl),None_ppl_ROI,131.657069,167.520333
245,None_0,GPT-3 FS-ICL (ppl),Pegasus-CD_ppl_ROI,131.657069,167.520333
246,None_0,GPT-3 ZS-ICL (ppl),None_ppl_ROI,47.182548,86.116629


In [171]:
# Split model by " "
# First is model, second is "exemplars", if third is Prompt Perplexity else Manual
def get_prompt_type(row):
    if '(ppl)' in row['model'].split(' '):
        return "Perplexity-optimized"
    else:
        return "Manual"

rotated_budget_df['model_fml'] = rotated_budget_df['model'].str.split(' ').str[0]
rotated_budget_df['Exemplars'] = rotated_budget_df['model'].str.split(' ').str[1]
rotated_budget_df['Prompt'] = rotated_budget_df.apply(get_prompt_type, axis=1)

rotated_budget_df.drop(columns=['model'], inplace=True)
rotated_budget_df.rename(columns={'model_fml': 'model'}, inplace=True)

In [172]:
# Split experiment by "_"
# First is Persona
rotated_budget_df['Persona'] = rotated_budget_df['experiment'].str.split('_').str[0]

## Save total budget to excel (Manish's format)

In [173]:
rotated_budget_df.to_excel(f'{output_dir}/{dataset_prefix}_budget.xlsx', index=False)

In [174]:
# split experiment by "_", first: Persona, second: Prompt, then drop
# if dataset_prefix == "MSC":
rotated_df['persona'] = rotated_df['experiment'].apply(lambda x: x.split('_')[0])

rotated_df['prompt'] = rotated_df['experiment'].apply(lambda x: x.split('_')[1])
rotated_df = rotated_df.drop(columns=['experiment'])

# Reorder columns. Metrics should come at the end
cols = rotated_df.columns.tolist()
non_metrics = [c for c in cols if c not in metrics]
rotated_df = rotated_df[non_metrics + metrics]

# Rename columns
rename_dict = {
    'prompt': 'Prompt',
    'persona': 'Persona',
    'model': 'Model'
}
rotated_df = rotated_df.rename(columns=rename_dict)

In [175]:
# write to excel. each model in a separate sheet
# rotated_df.to_excel(f"{output_dir}/merged.xlsx", index=False)
sheets = {
    'merged': rotated_df
}

# Separate sheet for each model
for m in models:
    m_df = rotated_df.loc[rotated_df['Model'] == m]
    # split experiment by "_", first: Persona, second: Prompt, then drop
    # m_df['persona'] = m_df['experiment'].apply(lambda x: x.split('_')[0])
    # m_df['prompt'] = m_df['experiment'].apply(lambda x: x.split('_')[1])
    # m_df = m_df.drop(columns=['experiment'])

    # remove index
    m_df = m_df.reset_index(drop=True)

    sheets[m] = m_df

    # output_path = os.path.join(output_dir, f'{m}.xlsx')
    # m_df.to_excel(output_path, index=False)

# Write to excel
with pd.ExcelWriter(f"{output_dir}/{dataset_prefix}_merged.xlsx") as writer:
    for sheet_name in sheets.keys():
        sheets[sheet_name].to_excel(writer, sheet_name=sheet_name, index=False, merge_cells=False)


In [176]:
rotated_df

metric,History Signal,Model,Persona,Prompt,Bert-f1,BLEU,Bleurt,DEB,METEOR,rougeL
0,BART,flanT5-XL FS-ICL,None,normal,0.024533,0.013192,0.181337,2.827780,-0.012220,-0.017885
1,BART,flanT5-XL FS-ICL,None,ppl,-1.284791,0.649517,2.342523,1.832087,1.632054,-0.564694
2,Full,flanT5-XL FS-ICL,None,normal,0.417784,0.266205,0.712758,3.268328,0.721557,-0.006995
3,Full,flanT5-XL FS-ICL,None,ppl,0.427779,0.279320,0.923966,3.425458,0.861939,0.039829
4,None_1,flanT5-XL FS-ICL,None,ppl,-6.931000,2.941883,9.561130,-30.052959,4.687062,-2.207612
...,...,...,...,...,...,...,...,...,...,...
203,Pegasus-CD,GPT-3 ZS-ICL,None,normal,-0.086305,-0.033461,0.052354,5.184283,1.639011,-0.284675
204,PegasusFT,GPT-3 ZS-ICL,None,normal,-0.060294,-0.123567,0.762640,3.414238,2.464788,-0.367376
205,PegasusFT,GPT-3 ZS-ICL,Pegasus-CD,normal,-0.152124,-0.149586,-0.596416,-2.602414,0.830964,-0.246360
206,Recent-4,GPT-3 ZS-ICL,None,normal,0.227191,-0.022504,0.728828,4.130606,1.423846,-0.131969


In [177]:
model_config = [m.split(' ') for m in models]

In [178]:
sheets['flanT5-XL FS-ICL']

metric,History Signal,Model,Persona,Prompt,Bert-f1,BLEU,Bleurt,DEB,METEOR,rougeL
0,BART,flanT5-XL FS-ICL,None,normal,0.024533,0.013192,0.181337,2.827780,-0.012220,-0.017885
1,BART,flanT5-XL FS-ICL,None,ppl,-1.284791,0.649517,2.342523,1.832087,1.632054,-0.564694
2,Full,flanT5-XL FS-ICL,None,normal,0.417784,0.266205,0.712758,3.268328,0.721557,-0.006995
3,Full,flanT5-XL FS-ICL,None,ppl,0.427779,0.279320,0.923966,3.425458,0.861939,0.039829
4,None_1,flanT5-XL FS-ICL,None,ppl,-6.931000,2.941883,9.561130,-30.052959,4.687062,-2.207612
5,None_1,flanT5-XL FS-ICL,Pegasus-CD,ppl,-0.425797,0.005420,-0.784206,-8.660981,-0.124925,-0.609587
6,Pegasus-CD,flanT5-XL FS-ICL,None,normal,-0.059001,-0.025982,-0.196230,4.986852,-0.182271,-0.207290
7,Pegasus-CD,flanT5-XL FS-ICL,None,ppl,-1.061798,0.648974,2.674944,4.820023,1.742553,-0.347252
8,PegasusFT,flanT5-XL FS-ICL,None,normal,0.007527,0.041281,0.064261,3.177177,0.064914,-0.016352
9,PegasusFT,flanT5-XL FS-ICL,None,ppl,-0.984420,0.634226,2.226776,2.112208,1.781234,-0.582146


In [179]:
sheets_v2 = {}
model_family = [m[0] for m in model_config]
for fam in model_family:
    zs_df = sheets[fam + " ZS-ICL"]
    fs_df = sheets[fam + " FS-ICL"]

    zs_df = zs_df.drop(columns=['Model'])
    fs_df = fs_df.drop(columns=['Model'])

    # if dataset_prefix=="MSC":
    zs_df = zs_df.set_index(['History Signal', 'Persona', 'Prompt'])
    fs_df = fs_df.set_index(['History Signal', 'Persona', 'Prompt'])
    # else:
    #     zs_df = zs_df.set_index(['History Signal', 'Persona', 'Prompt'])
    #     fs_df = fs_df.set_index(['History Signal', 'Persona', 'Prompt'])
    zs_fs_df = pd.concat([zs_df, fs_df], axis=1, keys=['ZS', 'FS']) 
    # sort by index in this order: Prompt, Persona, History Signal
    # if dataset_prefix=="MSC":
    zs_fs_df = zs_fs_df.sort_index(level=['Prompt', 'Persona', 'History Signal'])
    # else:
    #     zs_fs_df = zs_fs_df.sort_index(level=['Prompt', 'History Signal'])
    sheets_v2[fam] = zs_fs_df


# Write to excel
with pd.ExcelWriter(f"{output_dir}/{dataset_prefix}_merged_v2.xlsx", engine='xlsxwriter') as writer:
    for sheet_name in sheets_v2.keys():
        # Upto 4 decimal places
        sheets_v2[sheet_name].to_excel(writer, sheet_name=sheet_name, index=True, merge_cells=False, float_format="%.4f")

    # Apply color-scale to all sheets, each column
    if dataset_prefix == "MSC":
        start_col = 3
    else:
        start_col = 3

    for sheet_name in sheets_v2.keys():
        sheet = writer.sheets[sheet_name]
        for col in range(start_col, len(metrics) * 2 + 1 + start_col):
            # 3 color scale with midpoint at 0
            sheet.conditional_format(1, col, len(sheets_v2[sheet_name]), col, {
                'type': '3_color_scale',
                'min_color': '#ff6666',
                'mid_color': '#FFFFFF',
                'max_color': '#2eb82e',
                'mid_type': 'num',
                'mid_value': 0
            })


In [180]:
            # sheet.conditional_format(1, col, len(sheets_v2[sheet_name]), col, {
            #     'type': '2_color_scale',
            #     'min_color': '#FFFFFF',
            #     'max_color': '#00AA00'
            # })